In [1]:
import matplotlib
import numpy as np
import gym
import matplotlib.pyplot as plt
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from mpl_toolkits.mplot3d import Axes3D
import sklearn.pipeline
import sklearn.preprocessing
from scipy.special import logsumexp,softmax
import random

In [2]:
env = gym.make('MountainCar-v0')
num_episodes = 1000
discount_factor = 1.0
n_features = 50

nA = env.action_space.n


# Get satistics over observation space samples for normalization
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)

# Normalize and turn into feature
def featurize_state(state):
    # Transform data
    scaled = scaler.transform([state])
    featurized = featurizer.transform(scaled)
    return featurized

def Q(state,action,weight):
    value = state.dot(weight[action])
    return value
def max_Q(state,weight):
	value = np.max([Q(state,a,weight) for a in range(nA)])
	return value
def softmax_Q(state,weight,tau = 0.05):
    return logsumexp([Q(state,a,weight)/tau for a in range(nA)])

def greedy_policy(state, weight,epsilon=0.1):
    A = np.ones(nA,dtype=float) * epsilon/nA
    best_action =  np.argmax([Q(state,a,weight) for a in range(nA)])
    A[best_action] += (1.0-epsilon)
    sample = np.random.choice(nA,p=A)
    return sample

def greedy_policy_eva(state, weight):
    best_action =  np.argmax([Q(state,a,weight) for a in range(nA)])
    return best_action

def softmax_policy(state,weight,epsilon=0.1,tau = 0.05):
    A = np.ones(nA,dtype=float) * epsilon/nA
    A += (1.0-epsilon)*softmax([Q(state,a,weight)/tau for a in range(nA)]).flatten()
    sample = np.random.choice(nA,p=A)
    return sample

def softmax_policy_eva(state,weight,tau = 0.05):
    A = softmax([Q(state,a,weight)/tau for a in range(nA)]).flatten()
    sample = np.random.choice(nA,p=A)
    return sample

def policy_dist(state,action,weight,tau = 0.05):
    A = softmax([Q(state,a,weight)/tau for a in range(nA)]).flatten()
    return A[action]
def policy_grad(state,action,weight,tau =0.05):
    dist = policy_dist(state,action,weight,tau)
    return dist*state
def V(state,weight,tau = 0.05):
    return np.sum([Q(state,a,weight).dot(policy_dist(weight,a,weight,tau)) for a in range(nA)])
def max_double_Q(state,weight1,weight2):
    best_action =  np.argmax([Q(state,a,weight1) for a in range(nA)])
    value = Q(state,best_action,weight2)
    return value

### 150 Features

In [3]:
# Create radial basis function sampler to convert states to features for nonlinear function approx
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf", RBFSampler(gamma=1.0,random_state = 1, n_components=n_features))
        ])

# Fit featurizer to our scaled inputs
featurizer.fit(scaler.transform(observation_examples))

FeatureUnion(transformer_list=[('rbf',
                                RBFSampler(n_components=50, random_state=1))])

In [4]:
saved_model = {}
seed_range = np.arange(20)

#### Q-Learning

In [13]:
saved_model['Q-Learning'] = []
for seed in seed_range:
    np.random.seed(seed)
    alpha = 0.01
    w = np.zeros((nA,n_features))
    episode_rewards = np.zeros(num_episodes)
    for e in range(num_episodes):
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = greedy_policy(state,w)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            episode_rewards[e] += reward
            # Figure out target and td error
            target = reward + discount_factor * max_Q(next_state,w)		
            td_error = Q(state,action,w) - target
            # Find gradient with code to check it commented below (check passes)
            dw = (td_error).dot(state)
            # Update weight
            w[action] -= alpha * dw
            if done:
                break
            # update our state
            state = next_state
    saved_model['Q-Learning'].append(w)

#### Our Algorithm

In [14]:
alpha = 0.01
beta = 0.01
tau = 0.05
threshold = 500
def truncate(x,threshold):
    return threshold* np.tanh(x/threshold)
saved_model['Our Algorithm'] = []
for seed in seed_range:
    np.random.seed(seed)
    theta = np.zeros((nA,n_features))
    omega = np.zeros((nA,n_features))
    for e in range(num_episodes):
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = softmax_policy_eva(state,theta)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Update omega
            target = reward + discount_factor * truncate(tau*softmax_Q(next_state,theta),threshold)
            td_error = Q(state,action,omega)-target
            domega = td_error.dot(state)
            omega[action] -= beta*domega
            # Update theta
            Q_diff = Q(state,action,omega-theta)[0]
            grad_tanh = discount_factor * (1- np.square(truncate(tau*softmax_Q(next_state,theta),threshold))/(threshold**2))
    
            dtheta = (grad_tanh*policy_dist(next_state,action,theta)*next_state[0] - state[0])*Q_diff/np.linalg.norm(omega[action]-theta[action])
            theta[action] -= alpha*dtheta
            if done:
                break
            # update our state
            state = next_state
    saved_model['Our Algorithm'].append(theta)

#### Our Algorithm $\tau = 0.01$

In [15]:
alpha = 0.01
beta = 0.01
tau = 0.01
threshold = 500
def truncate(x,threshold):
    return threshold* np.tanh(x/threshold)
saved_model['Our Algorithm 0.01'] = []
for seed in seed_range:
    np.random.seed(seed)
    theta = np.zeros((nA,n_features))
    omega = np.zeros((nA,n_features))
    for e in range(num_episodes):
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = softmax_policy_eva(state,theta,tau = tau)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Update omega
            target = reward + discount_factor * truncate(tau*softmax_Q(next_state,theta,tau = tau),threshold)
            td_error = Q(state,action,omega)-target
            domega = td_error.dot(state)
            omega[action] -= beta*domega
            # Update theta
            Q_diff = Q(state,action,omega-theta)[0]
            grad_tanh = discount_factor * (1- np.square(truncate(tau*softmax_Q(next_state,theta,tau = tau),threshold))/(threshold**2))
    
            dtheta = (grad_tanh*policy_dist(next_state,action,theta,tau = tau)*next_state[0] - state[0])*Q_diff/np.linalg.norm(omega[action]-theta[action])
            theta[action] -= alpha*dtheta
            if done:
                break
            # update our state
            state = next_state
    saved_model['Our Algorithm 0.01'].append(theta)

#### Our Algorithm $\tau = 0.1$

In [16]:
alpha = 0.01
beta = 0.01
tau = 0.1
threshold = 500
def truncate(x,threshold):
    return threshold* np.tanh(x/threshold)
saved_model['Our Algorithm 0.1'] = []
for seed in seed_range:
    np.random.seed(seed)
    theta = np.zeros((nA,n_features))
    omega = np.zeros((nA,n_features))
    for e in range(num_episodes):
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = softmax_policy_eva(state,theta,tau = tau)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Update omega
            target = reward + discount_factor * truncate(tau*softmax_Q(next_state,theta,tau = tau),threshold)
            td_error = Q(state,action,omega)-target
            domega = td_error.dot(state)
            omega[action] -= beta*domega
            # Update theta
            Q_diff = Q(state,action,omega-theta)[0]
            grad_tanh = discount_factor * (1- np.square(truncate(tau*softmax_Q(next_state,theta,tau = tau),threshold))/(threshold**2))
    
            dtheta = (grad_tanh*policy_dist(next_state,action,theta,tau = tau)*next_state[0] - state[0])*Q_diff/np.linalg.norm(omega[action]-theta[action])
            theta[action] -= alpha*dtheta
            if done:
                break
            # update our state
            state = next_state
    saved_model['Our Algorithm 0.1'].append(theta)

#### Coupled Q-Learning

In [17]:
alpha = 0.01
beta = 0.01
saved_model['CQL'] = []
for seed in seed_range:
    np.random.seed(seed)
    u = np.zeros((nA,n_features))
    v = np.zeros((nA,n_features))
    for e in range(num_episodes):
    
        state = env.reset()[0]
        state = featurize_state(state)
    
        for j in range(200):
    
            #env.render()
            # Sample from our policy
            action = greedy_policy(state,v)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
    
            #u update
            du = Q(state,action,v).dot(state)-u[action]
            u[action] += alpha*du
            #v update
            target = reward + discount_factor * max_Q(next_state,u)		
            td = target -Q(state,action,v) 
            dv = td.dot(state)
            v[action] += beta*dv
            
            if done:
                break
            # update our state
            state = next_state
    saved_model['CQL'].append(v)

#### Double Q-Learning

In [18]:
alpha = 0.01
saved_model['DQL'] = []
for seed in seed_range:
    np.random.seed(seed)
    u = np.zeros((nA,n_features))
    v = np.zeros((nA,n_features))
    for e in range(num_episodes):
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
    
            #env.render()
            # Sample from our policy
            action = greedy_policy(state,u)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
    
            #Update
            rn = np.random.choice(2)
            if rn == 0: #update u
                target = reward + discount_factor * max_double_Q(next_state,u,v)
                td = target-Q(state,action,u)
                du = td.dot(state)
                u[action] += alpha* du
            if rn == 1: #update v
                target = reward + discount_factor * max_double_Q(next_state,v,u)
                td = target-Q(state,action,v)
                dv = td.dot(state)
                v[action] += alpha* dv
            if done:
                break
            # update our state
            state = next_state
    saved_model['DQL'].append(u)


#### TARGET NETWORK AND TRUNCATION

In [6]:
saved_model['Target_Truncation'] = []
for seed in seed_range:
    np.random.seed(seed)
    alpha = 0.01
    w = np.zeros((nA,n_features))
    u = w.copy()
    episode_rewards = np.zeros(num_episodes)
    for e in range(num_episodes):
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = greedy_policy(state,w)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            episode_rewards[e] += reward
            # Figure out target and td error
            target = reward + discount_factor * np.minimum(np.maximum(max_Q(next_state,u),-200),200)		
            td_error = Q(state,action,w) - target
            # Find gradient with code to check it commented below (check passes)
            dw = (td_error).dot(state)
            # Update weight
            w[action] -= alpha * dw
            if done:
                break
            if (j+1)% 20 ==0:
                u = w.copy()
            # update our state
            state = next_state
    saved_model['Target_Truncation'].append(w)

/home/jx3297/anaconda3/envs/torch_gpu/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


#### Evaluation

In [7]:
Performance = {}
n_traj_per_seed = 10

In [22]:
Performance['Q-Learning'] = []
np.random.seed(0)
random.seed(0)
for w in saved_model['Q-Learning']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = greedy_policy_eva(state,w)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['Q-Learning'].append(traj_reward)

In [40]:
np.mean(Performance['Q-Learning'])

-141.035

In [41]:
np.std(Performance['Q-Learning'])

24.765172622051313

In [25]:
Performance['Our Algorithm'] = []
tau = 0.05
np.random.seed(0)
random.seed(0)
for theta in saved_model['Our Algorithm']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = softmax_policy_eva(state,theta)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['Our Algorithm'].append(traj_reward)

In [26]:
np.mean(Performance['Our Algorithm'])

-120.165

In [27]:
np.std(Performance['Our Algorithm'])

20.641651460093982

In [28]:
tau = 0.01
Performance['Our Algorithm 0.01'] = []
np.random.seed(0)
random.seed(0)
for theta in saved_model['Our Algorithm 0.01']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = softmax_policy_eva(state,theta,tau =tau)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['Our Algorithm 0.01'].append(traj_reward)

In [29]:
np.mean(Performance['Our Algorithm 0.01'])

-139.58

In [30]:
np.std(Performance['Our Algorithm 0.01'])

20.58600495482307

In [31]:
tau = 0.1
Performance['Our Algorithm 0.1'] = []
np.random.seed(0)
random.seed(0)
for theta in saved_model['Our Algorithm 0.1']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = softmax_policy_eva(state,theta,tau =tau)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['Our Algorithm 0.1'].append(traj_reward)

In [32]:
np.mean(Performance['Our Algorithm 0.1'])

-131.56

In [33]:
np.std(Performance['Our Algorithm 0.1'])

28.28137903285481

In [34]:
Performance['CQL'] = []
np.random.seed(0)
random.seed(0)
for v in saved_model['CQL']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = greedy_policy_eva(state,v)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['CQL'].append(traj_reward)

In [35]:
np.mean(Performance['CQL'])

-200.0

In [36]:
np.std(Performance['CQL'])

0.0

In [37]:
Performance['DQL'] = []
np.random.seed(0)
random.seed(0)
for u in saved_model['DQL']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = greedy_policy_eva(state,u)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['DQL'].append(traj_reward)

In [42]:
np.mean(Performance['DQL'])

-155.595

In [43]:
np.std(Performance['DQL'])

19.43144294693526

In [8]:
Performance['Target_Truncation'] = []
np.random.seed(0)
random.seed(0)
for w in saved_model['Target_Truncation']:
    for k in range(n_traj_per_seed):
        traj_reward = 0
        state = env.reset()[0]
        state = featurize_state(state)
        for j in range(200):
            # Sample from our policy
            action = greedy_policy_eva(state,w)
            # Step environment and get next state and make it a feature
            next_state, reward, done, _,_ = env.step(action)
            next_state = featurize_state(next_state)
            # Statistic for graphing
            traj_reward += reward
            if done:
                break
            # update our state
            state = next_state
        Performance['Target_Truncation'].append(traj_reward)

In [9]:
np.mean(Performance['Target_Truncation'])

-140.46

In [10]:
np.std(Performance['Target_Truncation'])

26.304912088809573